In [5]:
import os
from openpyxl import *
import datetime

In [6]:
def get_call_num(ws, ws1, box_row, box_name):
    '''
    '''
    
    # iterate rows where current folder is to get call number
    for row in box_row:
        # extract the ark id
        ark_id = ws.cell(row = row, column = 1).value
        # extract folder number
        folder_name = ws.cell(row = row, column = 4).value
        # extract item number
        item_name = ws.cell(row = row, column = 6).value
        
        # output the information
        ws1.cell(row = row-box_row[0]+3 , column = 2).value = ark_id
        ws1.cell(row = row-box_row[0]+3 , column = 5).value = "200 b " + box_name + " f " + str(folder_name) + " i " + str(item_name)
        
        #print(ws1.cell(row = row-box_row[0]+3, column = 5).value)
        
    return ws1

In [7]:
def get_standard_ver(refer_list, field, value):
    '''this method is to get standard form of city, state and country 
    from reference, and return the standard form
    '''
    
    #create a dict to indicate the column number of 3 fields
    dict1 = dict(city = 1, state = 4, country = 7, venue = 10, date1 = 13, date2 = 15)
    
    # open location reference spreadsheet to get standard format
    wb1 = load_workbook(refer_list)
    ws1 = wb1["Sheet1"]

    # loop over reference spreadsheet
    for rows in ws1.iter_rows(min_row = 2, min_col = dict1[field], max_col = dict1[field], max_row = 1297):
        for cell in rows:
            # if find a matched a record, extract the standard version 
            if cell.value == value:
                value = ws1.cell(row = cell.row, column = dict1[field] + 1).value
                break
    
    return value

In [8]:
def handle_presumed(revis_list):
    '''
    '''
    
    # change None to a string to avoid: 'NoneType' is not iterable
    for i in range(0,3):
        if revis_list[i] == None:
            revis_list[i] = 'None'
    
    city = revis_list[0]
    state = revis_list[1]
    country = revis_list[2]
    
    # if more than one elements are presumed, put them in one []
    if '[' in city and '[' in state and '[' in country:
        revis_list[0] = city[:len(city)-1]
        revis_list[1] = state[1:len(city)-1]
        revis_list[2] = country[1:]
    elif '[' in city and '[' in state:
        revis_list[0] = city[:len(city)-1]
        revis_list[1] = state[1:]
    elif '[' in state and '[' in country:
        revis_list[1] = state[:len(city)-1]
        revis_list[2] = country[1:]
    elif '[' in city and '[' in country:
        revis_list[0] = city[:len(city)-1]
        revis_list[2] = country[1:]
    
    # change value back to None
    for i in range(0,3):
        if revis_list[i] == 'None':
            revis_list[i] = None
    
    return revis_list

In [9]:
def handle_combination(revis_list):
    '''get location combiantion
    '''
    
    # initialize location
    location = ''
    # add element to location
    for element in revis_list:
        if element != None:
            location = location + element + ', '
    
    # eliminate ',' at the end
    location = location[0:len(location)-2]
    
    return location

In [10]:
def get_location(ws, ws1, box_row, refer_list):
    '''
    '''
    
    # iterate rows where current folder is to get location
    for row in box_row:
        #get city
        city_orig = ws.cell(row = row, column = 9).value
        #get state
        state_orig = ws.cell(row = row, column = 10).value
        #get country
        country_orig = ws.cell(row = row, column = 11).value  
        # get original list
        orig_list = [city_orig, state_orig, country_orig]
        
        # try to know if three element are all missing
        all_none = True
        for element in orig_list:
            if element != '?' and element != '？' and element != None:
                all_none = False 
                
        # if city, state and country missing, make it 'unknown'
        if all_none:
            ws1.cell(row = row-box_row[0]+3, column = 12).value = 'unknown'
        # if not, get combination in correct format
        else:
            # get standard name
            city = get_standard_ver(refer_list, 'city', city_orig)
            state = get_standard_ver(refer_list, 'state', state_orig)
            country = get_standard_ver(refer_list, 'country', country_orig)
            # get revise list
            revis_list = [city, state, country]
            # handle presumed location
            handle_presumed(revis_list)     
            # get the final combination
            ws1.cell(row = row-box_row[0]+3, column = 12).value = handle_combination(revis_list)

        #print(ws1.cell(row = row-box_row[0]+3, column = 12).value)
        
    return ws1

In [11]:
def get_venue(ws, ws1, box_row, refer_list):
    '''
    '''
    
    # iterate rows where current folder is to get venue
    for row in box_row:
        
        venue_orig = ws.cell(row = row, column = 8).value
        # get standard name
        venue = get_standard_ver(refer_list, 'venue', venue_orig)
        
        # check none value
        if venue == None:
            ws1.cell(row = row-box_row[0]+3, column = 13).value = 'unknown'
        # fill in standard name
        else:
            ws1.cell(row = row-box_row[0]+3, column = 13).value = venue
        
    return ws1

In [12]:
def get_date(ws, ws1, box_row, refer_list):
    '''
    '''
    style1 = styles.NamedStyle(name='custom_datetime1', number_format='MM/YYYY HH:MM')
    #style1 = styles.Style(number_format="DD/MM/YYYY HH:MM:MM")
        
    for row in box_row:
        
        date_orig = ws.cell(row = row, column = 12).value
        # get standard date
        date1 = get_standard_ver(refer_list, 'date1', date_orig)
        date2 = get_standard_ver(refer_list, 'date2', date_orig)
        
        # check none value
        if date1 == None:
            ws1.cell(row = row-box_row[0]+3, column = 3).value = '[Concert program]'
            ws1.cell(row = row-box_row[0]+3, column = 9).value = 'unknown'
        # fill in standard name
        else:
            # if it is in datetime format, get the according string format
            if isinstance(date1, datetime.datetime):
                ws1.cell(row = row-box_row[0]+3, column = 3).value = '[Concert program ' + date1.strftime('%Y-%m-%d') + ']'
                ws1.cell(row = row-box_row[0]+3, column = 9).value = date2
                # set up cell style
                #ws.cell(row = row-box_row[0]+3, column = 9).style = style1
                #ws.cell(row = row-box_row[0]+3, column = 9).number_format = 'YYYY MMM DD'
                

            # if it is not in datetime format(with strange value), standard value will be enough
            else:
                ws1.cell(row = row-box_row[0]+3, column = 3).value = '[Concert program ' + date1 + ']'
                ws1.cell(row = row-box_row[0]+3, column = 9).value = date2
                
        #print(ws1.cell(row = row-box_row[0]+3, column = 3).value)
        #print(ws1.cell(row = row-box_row[0]+3, column = 9).value)
        
    return ws1

In [13]:
def get_page(ws, ws1, box_row):
    '''
    '''
    for row in box_row:
        
        # get page number and convert it to standard form
        page = ws.cell(row = row, column = 13).value
        
        # check none value
        if page != None:
            pagination = str(page) + ' p. ;  cm'
        else:
            pagination = '? p. ;  cm'
            
        # output to spreadsheet
        ws1.cell(row = row-box_row[0]+3, column = 7).value = pagination
    
    return ws1

In [14]:
def get_repeated_info(ws1, box_row):
    '''
    '''
    for row in box_row:
        
        # fill with repeated value
        ws1.cell(row = row-box_row[0]+3, column = 4).value = 'Marian Anderson Papers'
        ws1.cell(row = row-box_row[0]+3, column = 6).value = 'program'
        ws1.cell(row = row-box_row[0]+3, column = 23).value = 'For reproduction permission, contact XX'
        
    return ws1

In [15]:
def get_note(ws, ws1, box_row):
    '''
    '''
    for row in box_row:
        
        # get note
        note = ws.cell(row = row, column = 14).value
        ws1.cell(row = row-box_row[0]+3, column = 15).value = note
        
    return ws1

In [23]:
def etl(box_name, refer_list):
    '''
    '''
    
    # open xlsx with ark id; use data_only mode to exclude the formula
    wb = load_workbook\
         (r"/Users/zhiyuzhou/OneDrive/Penn/Work_on-campus/Marian_Anderson/Descriptive_matadata/test_data_4.3.19.xlsx", data_only=True)
    # get the first worksheet
    ws = wb["Sheet1"]
    
    # load descriptive metadata temple
    #wb1 = load_workbook\
         #(r"/Users/zhiyuzhou/OneDrive/Penn/18_Summer/Work_on-campus/Script/Descriptive_matadata/box_sheet/DM Ms. Coll 200V, Box " + box_name + ".xlsx", data_only=True)
    wb1 = load_workbook(r'/Users/zhiyuzhou/OneDrive/Penn/Work_on-campus/Marian_Anderson/Descriptive_matadata/box_sheet/DM MA CLIR 200V, template.xlsx')
    ws1 = wb1['Programs']

    # obtain box rows, put them in a list
    box_row = []

    # iterate box name column in xlsx
    for row in ws.iter_rows(min_row = 2, min_col = 3, max_col = 3, max_row = 196):
        for cell in row:
            # find rows where current box is, and record row number
            if str(cell.value) == box_name:
                #print (cell.value)
                box_row.append(cell.row)
    
    # eliminate first element
    box_row = box_row[1:]
    
    # updated by call number
    ws1 = get_call_num(ws, ws1, box_row, box_name)
    print('call number done...')
    
    # updated by location
    ws1 = get_location(ws, ws1, box_row, refer_list)
    print('location done...')
    
    # updated by venue
    ws1 = get_venue(ws, ws1, box_row, refer_list)
    print('venue done...')

    # updated by page
    ws1 = get_page(ws, ws1, box_row)
    print('page done...')
    
    # updated by repeated element
    ws1 = get_repeated_info(ws1, box_row)
    print('repeated done...')
    
    # updated by note
    ws1 = get_note(ws, ws1, box_row)
    print('note done...')
    
    # updated by date
    ws1 = get_date(ws, ws1, box_row, refer_list)
    print('date done...\n')
    
    wb1.save("/Users/zhiyuzhou/OneDrive/Penn/Work_on-campus/Marian_Anderson/Descriptive_matadata/box_sheet/DM Ms. Coll 200V, Box " + box_name + ".xlsx")

In [24]:
def main():
    
    refer_list = r"/Users/zhiyuzhou/OneDrive/Penn/Work_on-campus/Marian_Anderson/Descriptive_matadata/uniq_element_list_10.6.xlsx"
    
    for box in range (178, 179):
        
        print('box ' + str(box) + " data transfer start...\n")
        etl(str(box), refer_list)
        print('box ' + str(box) + " data transfer done...\n")
    
#     etl("182", refer_list)
#     print("data transfer done...\n")
                    
if __name__ == '__main__':
    main()


box 178 data transfer start...

date done...

box 178 data transfer done...

